# Data Struct

## Double List (include/linux/list.h)

```c
struct list_head {
	struct list_head *next, *prev;
};
```

1. The implementation is very simple. Put the `list_head` in your struct like

```c
struct semaphore {
	raw_spinlock_t		lock;
	unsigned int		count;
	struct list_head	wait_list;
};
```

and use `container_of` macro to get the container pointer. Very interesting.

------------------------



## 